<a href="https://colab.research.google.com/github/Zalina1276/Math-2024-25-Winter/blob/main/Notebooks_EN/01_Linear_Algebra/02_Systems_of_Linear_Equations/LA_SoLE_Gauss_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gaussian Elimination Method

Solving a system of linear equations using the Gaussian elimination method involves transforming the coefficient matrix of the system into an upper triangular matrix, followed by solving the system using back substitution.

This is one of the most fundamental methods for solving systems of linear equations, especially in cases where the number of unknowns is different from the number of equations.

In [ ]:
from sympy import Matrix, symbols, Eq, latex
from IPython.display import display, Markdown
import sympy as sp

class SymbolicMatrix:
    def __init__(self, matrix, column_labels=None, free_terms=None, display_mode="table"):
        self.matrix = Matrix(matrix)
        self.operations = []
        self.display_mode = display_mode

        if column_labels:
            if len(column_labels) != self.matrix.cols:
                raise ValueError("The number of column labels must be equal to the number of columns in the matrix.")
            self.column_labels = column_labels
        else:
            self.column_labels = [f"x{i+1}" for i in range(self.matrix.cols)]

        if free_terms:
            if len(free_terms) != self.matrix.rows:
                raise ValueError("The number of free terms must be equal to the number of rows in the matrix.")
            self.free_terms = free_terms
        else:
            self.free_terms = [0 for _ in range(self.matrix.rows)]

        display(Markdown("**Initial equations:**"))
        self.display_matrix()

    def set_display_mode(self, mode):
        if mode not in ["table", "equations"]:
            raise ValueError("Display mode must be 'table' or 'equations'.")
        self.display_mode = mode

    def display_matrix(self):
        if self.display_mode == "table":
            self._display_table()
        elif self.display_mode == "equations":
            self._display_equations()

    def _display_table(self):
        matrix_with_free_terms = self.matrix.row_join(Matrix(self.free_terms))
        headers = self.column_labels + ["Free term"]
        display(Markdown(self._matrix_to_markdown(matrix_with_free_terms, headers)))

    def _matrix_to_markdown(self, matrix, headers):
        markdown_str = "| " + " | ".join(headers) + " |\n"
        markdown_str += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        for i in range(matrix.rows):
            row = [str(matrix[i, j]) for j in range(matrix.cols)]
            markdown_str += "| " + " | ".join(row) + " |\n"
        return markdown_str

    def _display_equations(self):
        equations = []
        for i in range(self.matrix.rows):
            terms = []
            for j in range(self.matrix.cols):
                coef = self.matrix[i, j]
                var = symbols(self.column_labels[j])

                # Check if the coefficient is positive, negative, or zero
                if coef > 0:
                    if j == 0:
                        terms.append(f"{latex(coef)} {latex(var)}")
                    else:
                        terms.append(f"+ {latex(coef)} {latex(var)}")
                elif coef < 0:
                    terms.append(f"- {latex(abs(coef))} {latex(var)}")
                elif coef == 0:
                    if j == 0:
                        terms.append(f"0 {latex(var)}")
                    else:
                        terms.append(f"+0 {latex(var)}")

            lhs = " ".join(terms)
            rhs = latex(self.free_terms[i])

            equation = f"{lhs} = {rhs}"
            equations.append(equation)

        display(Markdown("System of equations:"))
        display(Markdown(f"$$\\begin{{aligned}} {self._equations_to_latex(equations)} \\end{{aligned}}$$"))

    def _equations_to_latex(self, equations):
        return " \\\\ ".join(equations)

    def add_row(self, target_row, source_row, coefficient):
        target_idx = self._validate_row_number(target_row)
        source_idx = self._validate_row_number(source_row)

        # Operation on the matrix
        self.matrix.row_op(target_idx, lambda v, j: v + coefficient * self.matrix[source_idx, j])

        # Operation on the free terms
        self.free_terms[target_idx] += coefficient * self.free_terms[source_idx]

        operation_str = f"r{target_row} = r{target_row} + {coefficient}*r{source_row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def multiply_row(self, row, coefficient):
        row_idx = self._validate_row_number(row)

        # Operation on the matrix
        self.matrix.row_op(row_idx, lambda v, _: coefficient * v)

        # Operation on the free terms
        self.free_terms[row_idx] *= coefficient

        operation_str = f"r{row} = {coefficient}*r{row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def swap_rows(self, row1, row2):
        row1_idx = self._validate_row_number(row1)
        row2_idx = self._validate_row_number(row2)

        self.matrix.row_swap(row1_idx, row2_idx)
        self.free_terms[row1_idx], self.free_terms[row2_idx] = self.free_terms[row2_idx], self.free_terms[row1_idx]

        operation_str = f"Swap r{row1} <-> r{row2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def swap_columns(self, col1, col2):
        col1_idx = self._validate_col_number(col1)
        col2_idx = self._validate_col_number(col2)

        self.matrix.col_swap(col1_idx, col2_idx)
        self.column_labels[col1_idx], self.column_labels[col2_idx] = self.column_labels[col2_idx], self.column_labels[col1_idx]

        operation_str = f"Swap col{col1} <-> col{col2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def _validate_row_number(self, row):
        if not isinstance(row, int):
            raise TypeError("Row number must be an integer.")
        if row < 1 or row > self.matrix.rows:
            raise IndexError(f"Row number must be between 1 and {self.matrix.rows}.")
        return row - 1

    def _validate_col_number(self, col):
        if not isinstance(col, int):
            raise TypeError("Column number must be an integer.")
        if col < 1 or col > self.matrix.cols:
            raise IndexError(f"Column number must be between 1 and {self.matrix.cols}.")
        return col - 1

    def print_operations(self):
        display(Markdown("**Performed operations:**"))
        for op in self.operations:
            print(op)

# Example usage:
m = SymbolicMatrix([[1,2,3], [4,5,6], [7,8,12]], column_labels=["x1", "x2", "x3"], free_terms=[10, 11, 18], display_mode="equations")


**Initial equations:**

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 4 x_{1} + 5 x_{2} + 6 x_{3} = 11 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [ ]:
m.add_row(2, 1, -4)

**Operation:** r2 = r2 + -4*r1

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [ ]:
m.add_row(3, 1, -7)

**Operation:** r3 = r3 + -7*r1

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} - 6 x_{2} - 9 x_{3} = -52 \end{aligned}$$

In [ ]:
m.add_row(3, 2, -2)

**Operation:** r3 = r3 + -2*r2

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} +0 x_{2} + 3 x_{3} = 6 \end{aligned}$$

In [ ]:
# We have 3 equations, so we end the operation
# Define symbols
x1, x2, x3 = symbols('x1 x2 x3')
# Define equations
eq1 = Eq(x1 + 2*x2 + 3*x3, 10)
eq2 = Eq(-3*x2 - 6*x3, -29)
eq3 = Eq(3*x3, 6)

In [ ]:
sol_x3 = sp.solve(eq3, x3)[0]
sol_x3

2

In [ ]:
# Substitute solution x3 into equation 2
eq2 = eq2.subs(x3, sol_x3)
# Solve equation 2
sol_x2 = sp.solve(eq2, x2)[0]
sol_x2

17/3

In [ ]:
# Substitute solutions x2 and x3 into equation 1
eq1 = eq1.subs({x2: sol_x2, x3: sol_x3})
# Solve equation 1
sol_x1 = sp.solve(eq1, x1)[0]
sol_x1

-22/3

---

## Exercises for Students

$$
\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {2 x+y+z=0} \\ {3 x+2 y+z=1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y+z-t=2} \\ {2 x+y+z=3} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y-z-t=0} \\ {2 x+3 y-2 z+t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.
$$

#***Homework***
##**Exercise1**
$\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {2 x+y+z=0} \\ {3 x+2 y+z=1}\end{array}\right.$

$1.R2=R2-2R1$

$\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {0-3y+5z=-8} \\ {3 x+2 y+z=1}\end{array}\right.$

$2.R3=R3-3R1$

$\left\{\begin{array}{c}{x+2y-2z=4} \\ {0-3y+5z=-8} \\ {0-4y+7z=-11}\end{array}\right.$

$3.R2=R2\cdot 4; R3=R3\cdot (-3)$

$\left\{\begin{array}{c}{x+2y-2z=4} \\ {0-12y+20z=-32} \\ {0+12y-21z=33}\end{array}\right.$

$4.R3=R3+R2$

$\left\{\begin{array}{c}{x+2y-2z=4} \\ {0-12y+20z=-32} \\ {0-0-1z=1}\end{array}\right.$

1)$-z=1$

$z=-1$

2)$-12y-20=-32$

$y=1$

3)$x+2+2=4$

$x=0$

Solution: x=0, y=1, z=-1

##**Exercise2**
$\left\{\begin{array}{c}{x+y+z-t=2} \\ {2 x+y+z=3} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.$

$R2=R2-2R1$

$\left\{\begin{array}{c}{x+y+z-t=2} \\ {0-y-z+2t=-1} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.$

$R3=R3+R1$

$\left\{\begin{array}{c}{x+y+z-t=2} \\ {0-y-z+2t=-1} \\ {0+y+2z-2t=2} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.$

$R4=R4-3R1$

$\left\{\begin{array}{c}{x+y+z-t=2} \\ {0-y-z+2t=-1} \\ {0+y+2z-2t=2} \\ {0-y-4z+5t=-7}\end{array}\right.$

$R3=R3+R2$

$\left\{\begin{array}{c}{x+y+z-t=2} \\ {0-y-z+2t=-1} \\ {0+0+z+0=1} \\ {0-y-4z+5t=-7}\end{array}\right.$

$R4=R4-R2$

$\left\{\begin{array}{c}{x+y+z-t=2} \\ {0-y-z+2t=-1} \\ {0+0+z+0=1} \\ {0-0-3z+3t=-6}\end{array}\right.$


1)$z=1$

2)$-3+3t=-6$

$t=-1$

3)$-y-1-2=-1$

$y=-2$

4)$x-2+1+1=2$

$x=2$

Solution:x=2, y=-2, z=1, t=-1

##**Exercise 3**
$\left\{\begin{array}{c}{x+y-z-t=0} \\ {2 x+3 y-2 z+t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.$

$1.R2=R2-2R1$

$\left\{\begin{array}{c}{x+y-z-t=0} \\ {0+y-0+3t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.$

$2.R3=R3-3R1$

$\left\{\begin{array}{c}{x+y-z-t=0} \\ {0+y-0+3t=4} \\ {0-3y+8z+3t=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.$

$R4=R4+R1$

$\left\{\begin{array}{c}{x+y-z-t=0} \\ {0+y-0+3t=4} \\ {0-3y+8z+3t=0} \\ {0+2y-4z+t=3}\end{array}\right.$

$R3=R3+3R1$

$\left\{\begin{array}{c}{x+y-z-t=0} \\ {0+y-0+3t=4} \\ {0-0+5z+0=0} \\ {0+2y-4z+t=3}\end{array}\right.$

$R4=R4-2R2$

$\left\{\begin{array}{c}{x+y-z-t=0} \\ {0+y-0+3t=4} \\ {0-0+5z+0=0} \\ {0+0-4z-5t=-5}\end{array}\right.$

1)$z=0$

2)$-5t=-5$

$t=1$

3)$y+3=4$

$y=1$

4)$x+1-0-1=0$

$x=0$

Solution: x=0, y=1, z=0, t=1

In [3]:
from sympy import symbols, Eq, solve

x, y, z = symbols('x, y, z')

# Define the equations based on the final simplified form
eq1 = Eq(x + 2*y - 2*z, 4)
eq2 = Eq(-12*y + 20*z, -32)
eq3 = Eq(-z, 1)

# Solve for the variables
sol_z = solve(eq3, z)[0]  # Solve for z first
eq2_sub = eq2.subs(z, sol_z)  # Substitute z into eq2
sol_y = solve(eq2_sub, y)[0]  # Solve for y

eq1_sub = eq1.subs({y: sol_y, z: sol_z})  # Substitute y and z into eq1
sol_x = solve(eq1_sub, x)[0]  # Solve for x

# Print the solutions
print(f"x = {sol_x}")
print(f"y = {sol_y}")
print(f"z = {sol_z}")

x = 0
y = 1
z = -1


In [1]:
from sympy import symbols, Eq, solve

x, y, z, t = symbols('x, y, z, t')

# Define the equations
eq1 = Eq(x + y + z - t, 2)
eq2 = Eq(-y - z + 2*t, -1)
eq3 = Eq(z, 1)  # Simplified from R3 = R3 + R2
eq4 = Eq(-y - 4*z + 5*t, -7)  # Assuming the last equation was cut off

# Solve for the variables
sol_z = solve(eq3, z)[0]  # Solve for z first
eq2_sub = eq2.subs(z, sol_z)  # Substitute z into eq2
eq4_sub = eq4.subs(z, sol_z)  # Substitute z into eq4

# Now we have two equations with two unknowns (y, t)
sol_y_t = solve((eq2_sub, eq4_sub), (y, t))
sol_y = sol_y_t[y]
sol_t = sol_y_t[t]

# Substitute y, z, t into eq1 to solve for x
eq1_sub = eq1.subs({y: sol_y, z: sol_z, t: sol_t})
sol_x = solve(eq1_sub, x)[0]

# Print the solutions
print(f"x = {sol_x}")
print(f"y = {sol_y}")
print(f"z = {sol_z}")
print(f"t = {sol_t}")

x = 2
y = -2
z = 1
t = -1


In [2]:
from sympy import symbols, Eq, solve

x, y, z, t = symbols('x, y, z, t')

# Define the equations based on the final simplified form
eq1 = Eq(x + y - z - t, 0)
eq2 = Eq(y + 3*t, 4)
eq3 = Eq(5*z, 0)  # This implies z = 0
eq4 = Eq(2*y - 4*z + t, 3)  # Assuming the last equation was cut off

# Solve for the variables
sol_z = solve(eq3, z)[0]  # Solve for z first
eq1_sub = eq1.subs(z, sol_z)  # Substitute z into eq1
eq2_sub = eq2.subs(z, sol_z)  # Substitute z into eq2 (no change in this case)
eq4_sub = eq4.subs(z, sol_z)  # Substitute z into eq4

# Now we have three equations with three unknowns (x, y, t)
sol_x_y_t = solve((eq1_sub, eq2_sub, eq4_sub), (x, y, t))
sol_x = sol_x_y_t[x]
sol_y = sol_x_y_t[y]
sol_t = sol_x_y_t[t]

# Print the solutions
print(f"x = {sol_x}")
print(f"y = {sol_y}")
print(f"z = {sol_z}")
print(f"t = {sol_t}")

x = 0
y = 1
z = 0
t = 1
